In [96]:
import uvicorn
import webbrowser
import nest_asyncio
from fastapi import FastAPI, HTTPException, Depends
from fastapi.security import OAuth2PasswordBearer
from jose import JWTError, jwt
from typing import List


In [97]:
#clave secreta y algoritmo de firma que necesita jwt
SECRET_KEY = "N4kyRQJA8J3eP4vZF0f7Md9rmQQFpfOg"
ALGORITHM = "HS256"

In [98]:
#Usuarios locales
users_db = {
    "usuario1": {"username": "user1", "password": "password123", "permissions": ["consume_api"], "disable": False},
    "usuario2": {"username": "user2", "password": "123password", "permissions": ["scrape_web"], "disable": False},
}

In [99]:
#Inicialización de usuarios
class User:
    def __init__(self, username: str, password: str, permissions: List[str]):
        self.username = username
        self.password = password
        self.permissions = permissions

In [100]:
#Crea el token
def create_jwt_token(data: dict):
    return jwt.encode(data, SECRET_KEY, algorithm=ALGORITHM)

In [101]:
#Instancia a FastAPI y su sistema de autenticación
app = FastAPI()
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="auth")

In [103]:
#Ruta para autenticación donde se valida que el usuario exista y la password esté correcta, también valida los scope que son los permisos de usuario
#si todo es correcto crea el token con la función previamente creada y devuelve un token de tipo bearer
@app.post("/auth")
async def authenticate_user(username: str, password: str):
    if username not in users_db or users_db[username]["password"] != password:
        raise HTTPException(status_code=401, detail="Credenciales inválidas")

    user = User(username=username, password=password, permissions=users_db[username]["permissions"])
    token_data = {"sub": username, "scopes": user.permissions}
    token = create_jwt_token(token_data)

    return {"token": token, "token_type": "bearer", "expires_in": 3600}

In [106]:
#Ejecuta el servidor
if __name__ == "__main__":
    #Se usa para problemas de concurrencia y ejecución del codigo correctamente
    nest_asyncio.apply()
    #Determina la ruta que se va a abrir en el navegador
    webbrowser.open("http://127.0.0.1:8000/docs")
    #Ejecuta la aplicación FastAPI
    uvicorn.run(app, host="127.0.0.1", port=8000)



INFO:     Started server process [26780]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:57297 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:57297 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:57300 - "POST /auth HTTP/1.1" 422 Unprocessable Entity


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [26780]
